In [ ]:
import pandas as pd
import numpy as np
import tensorly as ts
import urllib3
import json

In [ ]:
# Dataframe configs
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
# Setting SonarCloud.io project key
#chave = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd:packages&branch=master'

frontend_key = 'Guilherme-Aguiar_2020.1-VC_Gestor-FrontEnd&branch=master'
backend_key = 'Guilherme-Aguiar_2020.1-VC_Gestor-BackEnd&branch=master'

#backend
# https://sonarcloud.io/api/measures/component_tree?component=Guilherme-Aguiar_2020.1-VC_Gestor-BackEnd&branch=master&metricKeys=complexity,functions,files,comment_lines_density,duplicated_lines_density,coverage,sqale_rating,test_success_density,bugs,open_issues,reliability_rating,vulnerabilities,security_rating,ncloc,comment_lines
# frontend
# https://sonarcloud.io/api/measures/component_tree?component=Guilherme-Aguiar_2020.1-VC_Gestor-FrontEnd&branch=master&metricKeys=complexity,functions,files,comment_lines_density,duplicated_lines_density,coverage,sqale_rating,test_success_density,bugs,open_issues,reliability_rating,vulnerabilities,security_rating,ncloc,comment_lines

In [ ]:
# Defining the raw data extract from sonar cloud
metrics_list = ['files',
               'functions',
               'complexity',
               'ncloc',
               'comment_lines',
               'comment_lines_density',
               'duplicated_lines_density',
               'test_success_density',
               'coverage',
               'bugs',
               'reliability_rating',
               'vulnerabilities',
               'security_rating',
               'open_issues',
               'sqale_rating']

In [ ]:
print(len(metrics_list))

In [ ]:
# Assembling the metric list string and API request URLs

def generate_metric_string(metric_list):
    metric_str = ''

    for metric in metric_list:
        metric_str += metric + ','

    return metric_str

In [ ]:
metric_str = generate_metric_string(metrics_list)

In [ ]:
# Defining URLs

frontend_url = f"https://sonarcloud.io/api/measures/component_tree?component={frontend_key}&metricKeys={metric_str}&ps=500"
backend_url = f"https://sonarcloud.io/api/measures/component_tree?component={backend_key}&metricKeys={metric_str}&ps=500"

In [ ]:
# Starting connection
http = urllib3.PoolManager()

In [ ]:
# Sonar Cloud API Request
frontend_request = http.request('GET', frontend_url)
print(frontend_request.status)

backend_request = http.request('GET', backend_url)
print(backend_request.status)

In [ ]:
# Decoding json files extracted from sonar
frontend_json = json.loads(frontend_request.data.decode('utf-8'))
backend_json = json.loads(backend_request.data.decode('utf-8'))

In [ ]:
'''
# Saving json files locally
file_name = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd-R2.json'

with open(file_name, 'w') as outfile:
    json.dump(json_repository, outfile)
'''

def metric_per_file(json):
    file_json = []

    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)

    return file_json

In [ ]:
frontend_file_json = metric_per_file(frontend_json)
backend_file_json = metric_per_file(backend_json)

In [ ]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)

    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass

    df.reset_index(inplace = True)
    df.drop(['files'], axis=1)
    df = df.rename({'index': 'path'}, axis=1)

    return df

In [ ]:
frontend_files_data = generate_file_dataframe(metrics_list, frontend_file_json, language_extension = 'js')
backend_files_data = generate_file_dataframe(metrics_list, backend_file_json, language_extension = 'js')

In [ ]:
# FRONTEND METRICS DATA PER FILE

frontend_files_data

In [ ]:
# BACKEND METRICS DATA PER FILE

backend_files_data


In [ ]:
#printing CC per file descriptive statistics
backend_files_data['complexity'].astype(int).describe()

In [ ]:
backend_files_data['complexity/functions']=backend_files_data['complexity'].astype(float)/backend_files_data['functions'].astype(float)

#printing the average CC per function descriptive statistics
backend_files_data['complexity/functions'].describe()

In [ ]:
# Defining metrics for measure em1, where em1 = density of non-complex file
m0 = np.median(backend_files_data['complexity/functions'])
m1 = backend_files_data['complexity'].astype(float)
m2 = backend_files_data['functions'].astype(float)
m3 = len(backend_files_data)

#  intervals for em1 interpretation
x = np.array([0, m0])
y = np.array([1, 0])

# Defining intervals for em1 interpretation
IF1 = np.interp(list(((m1/m2) < m0)[(m2 > 0)]),x, y)

em1 = sum(IF1)/m3

em1

In [ ]:
# Defining metrics for measure em2, where em2 = density of commented files
m4 = backend_files_data['comment_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.3])
y = np.array([1, 0])

#(m4 >= 0.1 and m4 <= 0.3)

# Defining intervals for em2 interpretation
em2i = []
for if2i in m4:
    if 10 <= if2i <= 30:
        em2i.append(np.interp(if2i/100,x, y))
    else:
        em2i.append(0)

em2 = np.sum(em2i)/m3

em2

In [ ]:
# Defining metrics for measure ma3,where ma3 = absence of duplication on file (density)
m5 = backend_files_data['duplicated_lines_density'].astype(float)

#  intervals for em1 interpretation
x = np.array([0.1, 0.2])
y = np.array([1, 0])

# Intervals for ma3 interpretation
em3i = []
for if3i in m5:
    if 10 <= if3i <= 20:
        em3i.append(np.interp(if3i/100,x, y))
    else:
        em3i.append(0)

em3 = np.sum(em3i)/m3

em3

In [ ]:
# Defining  Modifiability quality sub-characteristic tensor.
# wem1 = 33%
# wem2 = 33%
# wem3 = 33%

wem1=wem2=wem3 = 0.3374

SC_Modifiability_EM_Weights = np.array([wem1, wem2, wem3]).reshape(1,3,1)
SC_Modifiability_EM = np.array([em1, em2, em3]).reshape(1,3,1)


T_SC_Modifiability = ts.tensor(SC_Modifiability_EM)


print("Modifiability Tensor" + '\n' + str(T_SC_Modifiability))
print(T_SC_Modifiability.ndim)

In [ ]:
# Weighting Modifiability tensor at measure level.

for i in range(T_SC_Modifiability.ndim):
     T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] = \
         ts.kron(T_SC_Modifiability[::T_SC_Modifiability.ndim][0][i-1] ,
                 SC_Modifiability_EM_Weights[::T_SC_Modifiability.ndim][0][i-1])

print("Weighted SC_Modifiability Tensor measures" + '\n' + str(T_SC_Modifiability))

In [ ]:
# Weighting Modifiability tensor at sub-characteristic level.
# wsc1 = 100%

wsc1 = 1

T_SC_Modifiability = ts.kron(T_SC_Modifiability,wsc1)

print("Weighted SC_Modifiability Tensor" + '\n' + str(T_SC_Modifiability))

In [ ]:
# Defining the maintainability quality characteristic tensor.

T_C_Maintainability = np.array([])
T_C_Maintainability = np.append(T_C_Maintainability,T_SC_Modifiability)

# Weighting Maintainability tensor at characteristic level.
# wc1 = 100%

wc1 = 1

T_C_Maintainability = ts.kron(T_C_Maintainability,wc1)

# Weighted C_Maintainability Tensor
print(str(T_C_Maintainability.reshape(1,3,1)))

In [ ]:
# Aggregating C_Maintainability Tensor. The aggregation view is obtained from Frobenius norm of tensor.
print(str(ts.norm(T_C_Maintainability)))

In [ ]:
SQC_1 = np.array([])
SQC_1 = np.append(SQC_1, T_C_Maintainability)

# The overall quality at Release 1 in VC_Gestor product.
# Github Repository: https://github.com/fga-eps-mds/2020.1-VC_Gestor-BackEnd
print(str(ts.norm(SQC_1)))
